<a href="https://colab.research.google.com/github/ronzus/HairClassifier/blob/main/HairClassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#hide
! [ -e /content ] && pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 719.8/719.8 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 40.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 43.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 77.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 39.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 40.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 47.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 49.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 21.7 MB/s eta 0:00:00
Mounted at /content/gdrive


In [27]:
#hide
from fastbook import *
from fastai.vision.widgets import *

So given a picture of hair ,we want to be able to say,
is it straight or curly.


How?
We just need to do the following basic steps:

1.Use Bing to search for images of "curly hair photos"

2.Use Bing to search for images of "straight hair photos"

3.Fine-tune a pretrained neural network to recognise these two groups

4.Try running this model on a picture of a hair and see if it works

In [8]:
#hide
key = os.environ.get('BING_SEARCH_V7_SUBSCRIPTION_KEY', '307341939e014bb4b0d38781e1eaff1d')
search_images_bing
resultsCurly = search_images_bing(key, 'curly hair')
resultsStraight = search_images_bing(key, 'straight hair')


ims1 = resultsCurly.attrgot('contentUrl')
ims2 = resultsStraight.attrgot('contentUrl')



In [9]:
#hide
hair_types = 'curly','straight'
path = Path('hair')

remove
if not path.exists():
    path.mkdir()
    for o in hair_types:
        dest = (path/o)
        dest.mkdir(exist_ok=True)
        results = search_images_bing(key, f'{o} hair')
        download_images(dest, urls=results.attrgot('contentUrl'))

In [10]:
#hide
fns = get_image_files(path)
failed = verify_images(fns)
failed.map(Path.unlink);

In [17]:
#hide
dls = DataBlock(
    blocks=(ImageBlock, CategoryBlock),
    get_items=get_image_files,
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y=parent_label,
    item_tfms=[Resize(192, method='squish')]
).dataloaders(path)


In [18]:
learn = vision_learner(dls, resnet18, metrics=error_rate)
learn.fine_tune(30)

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 167MB/s]


epoch,train_loss,valid_loss,error_rate,time
0,1.111123,1.677326,0.381818,00:17


epoch,train_loss,valid_loss,error_rate,time
0,0.367209,0.893880,0.327273,00:11
1,0.321019,0.453209,0.163636,00:11
2,0.255272,0.217990,0.090909,00:13
3,0.204087,0.121599,0.036364,00:11
4,0.166342,0.088097,0.054545,00:12
5,0.136777,0.097771,0.036364,00:13
6,0.114151,0.123025,0.054545,00:09
7,0.098552,0.155103,0.054545,00:13
8,0.085627,0.184818,0.054545,00:11
9,0.076648,0.199692,0.054545,00:10


In [23]:
learn.export()
path = Path()
path.ls(file_exts='.pkl')
learn_inf = load_learner(path/'export.pkl')

In [24]:
#hide
#hide_output
btn_upload = widgets.FileUpload()
out_pl = widgets.Output()
lbl_pred = widgets.Label()
btn_run = widgets.Button(description='Classify')



In [25]:
def on_click_classify(change):
    img = PILImage.create(btn_upload.data[-1])
    out_pl.clear_output()
    with out_pl: display(img.to_thumb(128,128))
    pred,pred_idx,probs = learn_inf.predict(img)
    lbl_pred.value = f'Prediction: {pred}; Probability: {probs[pred_idx]:.04f}'

btn_run.on_click(on_click_classify)

In [28]:
#hide_output
VBox([widgets.Label('Select your hair!'),
      btn_upload, btn_run, out_pl, lbl_pred])